### 導入 API KEY

In [ ]:
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv('api_key')

# api_key

### 測試讀取 API json格式

In [ ]:
import requests

url = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0016-001?Authorization=CWA-8BD90424-B8CA-46C4-A974-633DC4E1502D&format=JSON'
# url = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0015-001?Authorization=CWA-8BD90424-B8CA-46C4-A974-633DC4E1502D&limit=10&format=JSON&AreaName='
data = requests.get(url)   # 取得 JSON 檔案的內容為文字
data_json = data.json()    # 轉換成 JSON 格式
data_json

### 單獨讀取大/小規模地震資訊

In [ ]:
import requests

# 定義 API 請求的 URL
# url = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0015-001'
url = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0016-001'
params = {
    'Authorization': 'CWA-8BD90424-B8CA-46C4-A974-633DC4E1502D',
    # 'limit': 50,
    'format': 'JSON'
}

# 發送 API 請求
response = requests.get(url, params=params)

# 確認請求是否成功
if response.status_code == 200:
    data = response.json()
    earthquakes = data.get('records', {}).get('Earthquake', [])

    if earthquakes:
        for eq in earthquakes:
            print("=== 基本資料 ===")
            print(f"地震事件編號：{eq.get('EarthquakeNo', '無資料')}")
            print(f"報告類型：{eq.get('ReportType', '無資料')}")
            print(f"報告級別顏色：{eq.get('ReportColor', '無資料')}")
            print(f"報告內容：{eq.get('ReportContent', '無資料')}")
            print(f"報告圖片連結：{eq.get('ReportImageURI', '無資料')}")
            print(f"地震詳細連結：{eq.get('Web', '無資料')}")

            print("\n=== 地震資訊 ===")
            earthquake_info = eq.get('EarthquakeInfo', {})
            print(f"地震發生時間：{earthquake_info.get('OriginTime', '無資料')}")
            print(f"地震來源：{earthquake_info.get('Source', '無資料')}")
            print(f"震源深度：{earthquake_info.get('FocalDepth', '無資料')} 公里")

            epicenter = earthquake_info.get('Epicenter', {})
            print(f"震央位置描述：{epicenter.get('Location', '無資料')}")
            print(f"震央緯度：{epicenter.get('EpicenterLatitude', '無資料')}")
            print(f"震央經度：{epicenter.get('EpicenterLongitude', '無資料')}")

            magnitude = earthquake_info.get('EarthquakeMagnitude', {})
            print(f"規模類型：{magnitude.get('MagnitudeType', '無資料')}")
            print(f"規模值：{magnitude.get('MagnitudeValue', '無資料')}")

            # print("\n=== 震度資訊 ===")
            # intensity = eq.get('Intensity', {})
            # shaking_areas = intensity.get('ShakingArea', [])
            # for area in shaking_areas:
            #     print(f"\n地區描述：{area.get('AreaDesc', '無資料')}")
            #     print(f"縣市名稱：{area.get('CountyName', '無資料')}")
            #     print(f"地區震度：{area.get('AreaIntensity', '無資料')}")

            #     stations = area.get('EqStation', [])
            #     for station in stations:
            #         print(f"\n  測站名稱：{station.get('StationName', '無資料')}")
            #         print(f"  測站編號：{station.get('StationID', '無資料')}")
            #         print(f"  測站震度：{station.get('SeismicIntensity', '無資料')}")
            #         print(f"  測站緯度：{station.get('StationLatitude', '無資料')}")
            #         print(f"  測站經度：{station.get('StationLongitude', '無資料')}")
            #         print(f"  測站與震央距離：{station.get('EpicenterDistance', '無資料')} 公里")

            #         pga = station.get('pga', {})
            #         print(f"  PGA 東西方向：{pga.get('EWComponent', '無資料')}")
            #         print(f"  PGA 南北方向：{pga.get('NSComponent', '無資料')}")
            #         print(f"  PGA 垂直方向：{pga.get('VComponent', '無資料')}")
            #         print(f"  PGA 強度指標值：{pga.get('IntScaleValue', '無資料')}")

            #         pgv = station.get('pgv', {})
            #         print(f"  PGV 東西方向：{pgv.get('EWComponent', '無資料')}")
            #         print(f"  PGV 南北方向：{pgv.get('NSComponent', '無資料')}")
            #         print(f"  PGV 垂直方向：{pgv.get('VComponent', '無資料')}")
            #         print(f"  PGV 強度指標值：{pgv.get('IntScaleValue', '無資料')}")

            #         print(f"  地動波形圖連結：{station.get('WaveImageURI', '無資料')}")
            # print("\n" + "=" * 50 + "\n")
    else:
        print("無地震資料。")
else:
    print(f"請求失敗，HTTP 狀態碼：{response.status_code}")


### 合併大小規模地震資訊

In [ ]:
import requests
from datetime import datetime

def fetch_data(url, params):
    """
    從 API 獲取資料並解析 JSON。
    """
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get('records', {}).get('Earthquake', [])
    else:
        print(f"請求失敗，HTTP 狀態碼：{response.status_code}")
        return []

def merge_and_sort_desc(earthquake_list1, earthquake_list2):
    """
    合併兩個地震列表並按時間降序排列（越晚越上面）。
    """
    combined_list = earthquake_list1 + earthquake_list2

    # 按 OriginTime 排序（降序）
    combined_list.sort(key=lambda eq: datetime.strptime(
        eq.get('EarthquakeInfo', {}).get('OriginTime', '1970-01-01 00:00:00'),
        '%Y-%m-%d %H:%M:%S'
    ), reverse=True)

    return combined_list

def print_earthquake_info(earthquakes):
    """
    按順序輸出地震資料。
    """
    for eq in earthquakes:
        earthquake_info = eq.get('EarthquakeInfo', {})
        origin_time = earthquake_info.get('OriginTime', '無資料')
        magnitude = earthquake_info.get('EarthquakeMagnitude', {}).get('MagnitudeValue', '無資料')
        depth = earthquake_info.get('FocalDepth', '無資料')
        location = earthquake_info.get('Epicenter', {}).get('Location', '無資料')

        print(f"地震發生時間：{origin_time}")
        print(f"地震規模：{magnitude}")
        print(f"震源深度：{depth} 公里")
        print(f"震央位置：{location}")
        print("-" * 40)

# 定義兩個 API 的 URL 和參數
url1 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0016-001'
url2 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0015-001'

params = {
    'Authorization': 'CWA-8BD90424-B8CA-46C4-A974-633DC4E1502D',  
    'format': 'JSON'
}

# 獲取兩個 API 的資料
earthquake_list1 = fetch_data(url1, params)
earthquake_list2 = fetch_data(url2, params)

# 合併並按時間降序排序
sorted_earthquakes = merge_and_sort_desc(earthquake_list1, earthquake_list2)

# 輸出結果
print("=== 合併並按時間降序排列的地震資料 ===")
print_earthquake_info(sorted_earthquakes)


### 加入絕對位置

In [ ]:
import requests
from datetime import datetime

def fetch_data(url, params):
    """
    從 API 獲取資料並解析 JSON。
    """
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        return data.get('records', {}).get('Earthquake', [])
    else:
        print(f"請求失敗，HTTP 狀態碼：{response.status_code}")
        return []

def merge_and_sort_desc(earthquake_list1, earthquake_list2):
    """
    合併兩個地震列表並按時間降序排列（越晚越上面）。
    """
    combined_list = earthquake_list1 + earthquake_list2

    # 按 OriginTime 排序（降序）
    combined_list.sort(key=lambda eq: datetime.strptime(
        eq.get('EarthquakeInfo', {}).get('OriginTime', '1970-01-01 00:00:00'),
        '%Y-%m-%d %H:%M:%S'
    ), reverse=True)

    return combined_list

def print_earthquake_info(earthquakes):
    """
    按順序輸出地震資料，包括經緯度。
    """
    for eq in earthquakes:
        earthquake_info = eq.get('EarthquakeInfo', {})
        origin_time = earthquake_info.get('OriginTime', '無資料')
        magnitude = earthquake_info.get('EarthquakeMagnitude', {}).get('MagnitudeValue', '無資料')
        depth = earthquake_info.get('FocalDepth', '無資料')
        epicenter = earthquake_info.get('Epicenter', {})
        location = epicenter.get('Location', '無資料')
        latitude = epicenter.get('EpicenterLatitude', '無資料')
        longitude = epicenter.get('EpicenterLongitude', '無資料')

        print(f"地震發生時間：{origin_time}")
        print(f"地震規模：{magnitude}")
        print(f"震源深度：{depth} 公里")
        print(f"震央位置：{location}")
        print(f"震央經度：{longitude}")
        print(f"震央緯度：{latitude}")
        print("-" * 40)

# 定義兩個 API 的 URL 和參數
url1 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0016-001'
url2 = 'https://opendata.cwa.gov.tw/api/v1/rest/datastore/E-A0015-001'

params = {
    'Authorization': 'CWA-8BD90424-B8CA-46C4-A974-633DC4E1502D',  
    'format': 'JSON'
}

# 獲取兩個 API 的資料
earthquake_list1 = fetch_data(url1, params)
earthquake_list2 = fetch_data(url2, params)

# 合併並按時間降序排序
sorted_earthquakes = merge_and_sort_desc(earthquake_list1, earthquake_list2)

# 輸出結果
print("=== 合併並按時間降序排列的地震資料 ===")
print_earthquake_info(sorted_earthquakes)
